# Final Project - PHY 256

### Serena Flint

### Data Preparation

##### Reading Data from .CSV Files

In [44]:
import csv

#CSV file name to be read in
file = "TrainData.csv"

fields = []
rows = []

#adds file data to rows[]
with open(file, 'r') as csvfile:
    csvreader = csv.reader(csvfile)  
    for row in csvreader:
        rows.append(row)
        
    #this is the limiting number of training cases from known galaxy classifications
    print("Total number of rows:", csvreader.line_num)

print("\nArray Formatting:")
print(rows[0])

csvfile.close()

Total number of rows: 61579

Array Formatting:
['GalaxyID', 'Class1.1', 'Class1.2', 'Class1.3', 'Class2.1', 'Class2.2', 'Class3.1', 'Class3.2', 'Class4.1', 'Class4.2', 'Class5.1', 'Class5.2', 'Class5.3', 'Class5.4', 'Class6.1', 'Class6.2', 'Class7.1', 'Class7.2', 'Class7.3', 'Class8.1', 'Class8.2', 'Class8.3', 'Class8.4', 'Class8.5', 'Class8.6', 'Class8.7', 'Class9.1', 'Class9.2', 'Class9.3', 'Class10.1', 'Class10.2', 'Class10.3', 'Class11.1', 'Class11.2', 'Class11.3', 'Class11.4', 'Class11.5', 'Class11.6']


##### Binning


In [46]:
#desired output path
output = open("bins.txt", "w")

line_arr1 = []
line_arr2 = []
line_arr3 = []



#possible bins
bin1A = [] #elliptical
bin1B = [] #spiral
bin2A = [] #bar/no bar
bin2B = [] #bar/no bar

for i in range (1, 1001):
    if rows[i][1] > rows[i][2]:
        bin1A.append(rows[i][0])
    else:
        if rows[i][4] > rows[i][5]:
            bin2A.append(rows[i][0])
        else:
            bin2B.append(rows[i][0])

print("Number Elliptical:", len(bin1A))
print("Number Spiral A:", len(bin2A))
print("Number Spiral B:", len(bin2B))

#for i in range (1, 1000):
  #  line_str = rows[i][0] + "," + rows[i][1] + "," + rows[i][2] + '\n'
  #  line_arr.append(line_str)
    #print(line_str) #sanity check
    
#writes formatted data to desired output path
#output.writelines(line_arr)

output.close()

Number Elliptical: 429
Number Spiral A: 109
Number Spiral B: 462


### Algorithm & Clustering

##### Imports

In [40]:
import cv2 
import os 
import numpy as np 
from random import shuffle 
from tqdm import tqdm 

import tflearn 
from tflearn.layers.conv import conv_2d, max_pool_2d 
from tflearn.layers.core import input_data, dropout, fully_connected 
from tflearn.layers.estimator import regression 
  
import tensorflow as tf 

import matplotlib.pyplot as plt 

ModuleNotFoundError: No module named 'cv2'